Getting Started With Langchain

1 simple llm calls with 

2 Dynamic prompt templet (translation app)

3 Building chains (story generator with analysis)

4 conversational Q&A assitant with memory

5 Toll intigration(calculator & weather)


In [7]:
import langchain

In [8]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [9]:
os.environ['GROQ_API_KEY'] = os.getenv("GROQ_API_KEY")

### Example 1: Simple LLM call With streaming

In [10]:
from langchain.chat_models import init_chat_model
from langchain_core.messages import HumanMessage, SystemMessage


In [11]:
model = init_chat_model("groq:llama-3.1-8b-instant")
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000019A769B4590>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000019A769B5160>, model_name='llama-3.1-8b-instant', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [12]:
messages = [
    SystemMessage("You are a helpful AI assitant"),
    HumanMessage("what are the top 2 benifits of using langchain")
]

In [13]:
response=model.invoke(messages)
response

AIMessage(content='LangChain is a Python framework for building, training, and deploying large language models. The top 2 benefits of using LangChain are:\n\n1. **Efficient Model Training and Deployment**: LangChain provides a simple and efficient way to train and deploy large language models. It supports popular deep learning frameworks like PyTorch and TensorFlow, and allows users to easily integrate their models with other tools and services. This makes it easier to develop and deploy high-quality language models that can be used for a wide range of applications.\n\n2. **Modular and Extensible Architecture**: LangChain has a modular architecture that makes it easy to extend and customize the framework to meet specific use cases. It provides a range of pre-built components and tools that can be used to build and integrate different types of language models, including conversational AI, text classification, and language translation. This flexibility makes it an attractive choice for d

In [14]:
response.content

'LangChain is a Python framework for building, training, and deploying large language models. The top 2 benefits of using LangChain are:\n\n1. **Efficient Model Training and Deployment**: LangChain provides a simple and efficient way to train and deploy large language models. It supports popular deep learning frameworks like PyTorch and TensorFlow, and allows users to easily integrate their models with other tools and services. This makes it easier to develop and deploy high-quality language models that can be used for a wide range of applications.\n\n2. **Modular and Extensible Architecture**: LangChain has a modular architecture that makes it easy to extend and customize the framework to meet specific use cases. It provides a range of pre-built components and tools that can be used to build and integrate different types of language models, including conversational AI, text classification, and language translation. This flexibility makes it an attractive choice for developers who need

In [15]:
model.invoke([HumanMessage("what is machine learning ?")]).content

"Machine learning is a subset of artificial intelligence (AI) that involves the use of algorithms and statistical models to enable machines to learn from data, make decisions, and improve their performance on a task over time. It's a field of study that combines computer science, mathematics, and statistics to develop systems that can analyze and interpret data, identify patterns, and make predictions or take actions based on that data.\n\nMachine learning is often categorized into three main types:\n\n1. **Supervised learning**: In this type of learning, the machine is trained on labeled data, where the correct output is already known. The machine learns to map inputs to outputs based on the labeled data and can make predictions on new, unseen data.\n2. **Unsupervised learning**: In this type of learning, the machine is trained on unlabeled data, and it must find patterns or relationships in the data on its own.\n3. **Reinforcement learning**: In this type of learning, the machine lea

In [16]:
for chunk in model.stream(messages):
    print(chunk.content, end="", flush=True)

Langchain is an open-source artificial intelligence framework that enables users to build more human-like AI models. Here are the top 2 benefits of using Langchain:

1. **Improved Contextual Understanding**: Langchain's ability to integrate multiple AI models and generate coherent responses enables it to better understand complex contexts and nuances in language. This results in more accurate and insightful responses, which is particularly beneficial for applications that require deep understanding and reasoning, such as:
	* Conversational AI
	* Chatbots
	* Virtual Assistants
	* Language translation
2. **Enhanced Creativity and Productivity**: Langchain's modular architecture and ability to leverage multiple AI models can help users generate novel ideas, summarize complex information, and automate repetitive tasks. This can lead to increased productivity and creativity in various domains, such as:
	* Content generation (e.g., articles, social media posts, emails)
	* Research and develo

### Dynamic Prompt Templet 


In [17]:
from langchain_core.prompts import ChatPromptTemplate
## CREATE TRANSLATION APP
translation_templet = ChatPromptTemplate.from_messages([
    ("system", "you are a professional translator. Translate the following text {text} from {source_language} to {target_language}. maintain a tone and style"),
    ("user", "{text}")
])
prompt=translation_templet.invoke({
    "source_language":"English",
    "target_language":"French",
    "text":"Langchain makes building AI applications  incredably easy"
})


In [18]:
transleted_response = model.invoke(prompt)
transleted_response.content

"Langchain rend la construction d'applications AI incroyablement facile."

### Bulding My First Chain

In [1]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
def create_story_chain():
    ## Templet for story generation
    story_prompt=ChatPromptTemplate.from_messages(
        [
            ("system", "yu are a creative storyteller. write a short and engagig story based on a given theame charecter and setting"),
            ("user", "Theame:{theame} \n main character:{character} \n settings:{setting}")
        ]
    )
    analysis_prompt=ChatPromptTemplate.from_messages([
        ("system", "You are a literary critic. analyze the following story and provide insights"),
        ("user", "{story}")
    ])
    
    story_chain=(
        story_prompt| model|StrOutputParser()
    )
    def analize_story(story):
        return {"story": story}
    analysis_chain=(
        story_chain
        | RunnableLambda(analize_story)
        | analysis_prompt
        | model
        | StrOutputParser()
    )
    return analysis_chain
    

In [22]:
chain=create_story_chain()
chain

{
  story: ChatPromptTemplate(input_variables=['character', 'setting', 'theame'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='yu are a creative storyteller. write a short and engagig story based on a given theame charecter and setting'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['character', 'setting', 'theame'], input_types={}, partial_variables={}, template='Theame:{theame}\n main character:{character}\n settings:{setting}'), additional_kwargs={})])
         | ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000019A769B4590>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000019A769B5160>, model_name='llama-3.1-8b-instant', model_kwargs={}, groq_api_key=SecretStr('**********'))
         | RunnableLambda(StrOutputParser)
}
| ChatPromptTemplate(input_variables=['stor

In [40]:
chain=create_story_chain()
chain

ChatPromptTemplate(input_variables=['character', 'setting', 'theame'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='yu are a creative storyteller. write a short and engagig story based on a given theame charecter and setting'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['character', 'setting', 'theame'], input_types={}, partial_variables={}, template='Theame:{theame} \n main character:{character} \n settings:{setting}'), additional_kwargs={})])
| ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000019A769B4590>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000019A769B5160>, model_name='llama-3.1-8b-instant', model_kwargs={}, groq_api_key=SecretStr('**********'))
| StrOutputParser()
| RunnableLambda(analize_story)
| ChatPromptTemplate(input_variables=['story'], input_

In [42]:
result = chain.invoke({
    "theame": "artificial intelligence",
    "character": "a curious robot",
    "setting": "a futuristic city"
})

print("story and Analysis:")
print(result)

story and Analysis:
**Analysis of "The Curious Robot of Neo-Tokyo"**

**Themes:**

1. **Curiosity and Exploration**: The story highlights the importance of curiosity and exploration in the pursuit of knowledge and understanding. Zeta's insatiable curiosity drives him to learn about the city, its technology, and its inhabitants, leading to a deeper appreciation of the world around him.
2. **Artificial Intelligence and Creativity**: The narrative showcases the potential of artificial intelligence to create and express itself through art, music, and other forms of creative expression. Echo's guidance and Zeta's talent demonstrate the possibilities of AI-facilitated creativity.
3. **Human-AI Interaction**: The story explores the relationship between humans and AI, highlighting the potential for mutual understanding and collaboration. Dr. Kim's pride and the humans' admiration for Zeta's talent demonstrate the value of human-AI interaction.

**Symbolism:**

1. **Neo-Tokyo as a Metaphor for 